In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler

import wandb

In [2]:
train_path = "C:/Users/navee/OneDrive/Documents/dl/A2/inaturalist_12K/train"
test_path = "C:/Users/navee/OneDrive/Documents/dl/A2/inaturalist_12K/val"
def load_data(train_path, val_split=0.2, batch_size=150, data_augmentation=False):
    transform_list = [transforms.Resize((224, 224)),transforms.ToTensor(),
                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
    if data_augmentation:
        transform_list.insert(0, transforms.RandomHorizontalFlip())
        transform_list.insert(0, transforms.RandomRotation(10))

    transform = transforms.Compose(transform_list)

    # Load dataset
    dataset = ImageFolder(train_path, transform=transform)
    num_classes = len(dataset.classes)
    
    # Create stratified train-validation split
    train_indices, val_indices = train_test_split(list(range(len(dataset))), 
                                                   test_size=val_split, 
                                                   stratify=dataset.targets)

    # Create DataLoader for training set
    train_sampler = SubsetRandomSampler(train_indices)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)

    # Create DataLoader for validation set
    val_sampler = SubsetRandomSampler(val_indices)
    val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

    return train_loader, val_loader, num_classes

In [ ]:
!wandb login "8506a95d562352ae82e224bf1d33b8cdf34c0c0e" 

In [4]:

train_loader, val_loader, num_classes = load_data(train_path, val_split=0.2, batch_size=32, data_augmentation=True)
# Load pre-trained ResNet50 model
pretrained_resnet50 = models.resnet50(pretrained=True)

# Freeze all convolutional layers
for param in pretrained_resnet50.parameters():
    param.requires_grad = False

# Replace the classifier with a new fully connected layer

pretrained_resnet50.fc = nn.Linear(pretrained_resnet50.fc.in_features, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(pretrained_resnet50.fc.parameters(), lr=learning_rate)

# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pretrained_resnet50.to(device)

# Train the model
num_epochs = 10
wandb.init()
for epoch in range(num_epochs):
    # Training loop
    pretrained_resnet50.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move tensors to GPU
        optimizer.zero_grad()
        outputs = pretrained_resnet50(inputs)
        train_loss = criterion(outputs, labels)
        train_loss.backward()
        optimizer.step()
        
        running_loss += train_loss.item() * inputs.size(0)
        
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    avg_train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = correct_train / total_train
    
    # Validation loop (evaluate model performance on validation dataset)
    pretrained_resnet50.eval()
    validation_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move tensors to GPU
            outputs = pretrained_resnet50(inputs)
            val_loss = criterion(outputs, labels)
            val_loss.backward()
            
            validation_loss += val_loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    validation_accuracy = correct_val / total_val
    avg_val_loss = validation_loss / len(val_loader.dataset)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {avg_train_loss:.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, '
          f'Validation Loss: {avg_val_loss:.4f}'
          f'Validation Accuracy: {validation_accuracy:.4f}')
    
    run_name = "{}_lr{}".format("adam", learning_rate)
    wandb.run.name = run_name
    wandb.log({"epoch": epoch+1, 
                "train_loss": avg_train_loss, 
                "train_accuracy": train_accuracy, 
                "val_loss": avg_val_loss, 
                "val_accuracy": validation_accuracy})

c:\Users\navee\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\navee\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Train Loss: 0.9481, Train Accuracy: 0.6137, Validation Accuracy: 0.7140
Epoch [2/10], Train Loss: 0.6811, Train Accuracy: 0.7205, Validation Accuracy: 0.7280
Epoch [3/10], Train Loss: 0.6596, Train Accuracy: 0.7253, Validation Accuracy: 0.7250
Epoch [4/10], Train Loss: 0.6377, Train Accuracy: 0.7342, Validation Accuracy: 0.7425
Epoch [5/10], Train Loss: 0.6249, Train Accuracy: 0.7422, Validation Accuracy: 0.7410
Epoch [6/10], Train Loss: 0.6174, Train Accuracy: 0.7452, Validation Accuracy: 0.7405
Epoch [7/10], Train Loss: 0.5943, Train Accuracy: 0.7507, Validation Accuracy: 0.7270
Epoch [8/10], Train Loss: 0.6016, Train Accuracy: 0.7501, Validation Accuracy: 0.7405
Epoch [9/10], Train Loss: 0.5857, Train Accuracy: 0.7553, Validation Accuracy: 0.7515
Epoch [10/10], Train Loss: 0.5837, Train Accuracy: 0.7567, Validation Accuracy: 0.7370
